In [1]:
import datetime
from pyspark.sql.functions import rand, randn, lit, col



In [2]:
#Generate some data per day
#numdays=400
#base = datetime.datetime.strptime("2017-10-23","%Y-%m-%d")
#date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
#ugly way of generating data with For loop, Write away per Y/M/D
# for date in date_list:
  
#   temp_df = sqlContext.range(0, 10)
#   temp_date=date
# # Generate two other columns using uniform distribution and normal distribution.
#   temp_df=temp_df.select("id", lit(temp_date).alias("date"), randn(seed=27).alias("data"))
#   temp_df.write.format('com.databricks.spark.csv') \
#   .mode('overwrite').option("header", "true")\
#   .save("/mnt/partitioning/new/"+str(temp_date.year)+"/"+str(temp_date.month)+"/"+str(temp_date.day)+"/test.csv")
  


In [3]:
#Read simulated data to detect schema
df=spark.read.format('csv').options(header='true', inferSchema='true').load("mnt/partitioning/new/2019/1/1/*.csv")

#define Schema
df_schema=df.schema

In [4]:
startdate="2016-11-30"
enddate="2019-2-26"

In [5]:
#Function to read data efficiently from subdirectories. Seems to be faster than load.where() approach, 
def read_efficient(startdate,enddate,directory,schema):
 #Construct Year Range
  yearrange=range(datetime.datetime.strptime(startdate,"%Y-%m-%d").year\
                  ,datetime.datetime.strptime(enddate,"%Y-%m-%d").year+1)
  #Extract month of startdate and enddate
  [minmonth,maxmonth]=[datetime.datetime.strptime(startdate,"%Y-%m-%d").month,\
                       datetime.datetime.strptime(enddate,"%Y-%m-%d").month]
  #Extract day of startdate and enddate
  [minday,maxday]= [datetime.datetime.strptime(startdate,"%Y-%m-%d").day,datetime.datetime.strptime(enddate,"%Y-%m-%d").day]
  #Construct 5 seperate dataframes (if existing): 
  # - Rest of days of month of startdate
  # - Rest of months of year of startdate
  # - Middle Full years
  # - Rest of months of year of enddate
  # - Rest of days of last month of enddate
  
  #Remaining days of month of startdate:
  temp_begin_day=spark.read.format('csv').\
    options(header='true',schema=schema).\
    load(str(directory)+str(list(yearrange)[0])+"/"+str(minmonth)+"/"+'{' + ','.join(map(str, list(range(minday,32)))) + '}'+"/*.csv")
  
  #If statement to check if there are full months left of year of startdate, if not, pass empty DF
  if minmonth!=12:
    temp_begin_month=spark.read.format('csv').\
      options(header='true',schema=schema).\
      load(str(directory)+str(list(yearrange)[0])+"/"+'{' + ','.join(map(str, list(range(minmonth+1,13)))) + '}'+"/*/*.csv")
  else:
    temp_begin_month=spark.createDataFrame([], schema)
  
  #If statement to check if there are full months left of year of enddate, if not, pass empty DF
  if maxmonth !=1:
    temp_end_month=spark.read.format('csv').\
      options(header='true',schema=schema).\
      load(str(directory)+str(list(yearrange)[-1])+"/"+'{' + ','.join(map(str, list(range(1,maxmonth)))) + '}'+"/*/*.csv")
  else:
    temp_end_month=spark.createDataFrame([], schema)
    
  #Remaining days of month of enddate:  
  temp_end_day=spark.read.format('csv').\
    options(header='true',schema=df_schema).\
    load(str(directory)+str(list(yearrange)[-1])+"/"+str(maxmonth)+"/"+'{' + ','.join(map(str, list(range(1,maxday)))) + '}'+"/*.csv")
  
  #If statement to check if there are full years between year startdate and year enddate, if not, pass empty DF
  if list(yearrange)[1:-1]!=[]:
    temp_middle=spark.read.format('csv').\
      options(header='true',schema=df_schema).\
      load(str(directory)+'{' + ','.join(map(str, list(yearrange[1:-1]))) + '}'+"/*/*/*.csv")
    return(temp_begin_day.union(temp_begin_month.union(temp_middle.union(temp_end_month.union(temp_end_day)))))
  else:
    return(temp_begin_day.union(temp_begin_month.union(temp_end_month.union(temp_end_day))))


In [6]:
df=read_efficient("2018-12-03","2019-01-12","mnt/partitioning/new/")

2018-12-03 2019-01-12
12 1
3 12

In [7]:
from pyspark.sql.functions import year, count
(df
    .groupBy(year("date").alias("year"))
    .agg(count("data").alias("count"))
    .show())

+----+-----+
year|count|
+----+-----+
2018| 290|
2019| 110|
+----+-----+

In [8]:
#Different approach to read all, then filter

spark.read.format('csv').options(header='true',schema=df_schema).load("mnt/partitioning/new/*/*/*/*.csv").\
  where(col('date').between(datetime.datetime.strptime(startdate,"%Y-%m-%d"), datetime.datetime.strptime(enddate,"%Y-%m-%d"))).explain()

== Physical Plan ==
*(1) Project [id#30839, date#30840, data#30841]
+- *(1) Filter ((isnotnull(date#30840) && (date#30840 >= 2017-10-23 00:00:00)) && (date#30840 <= 2019-11-18 00:00:00))
 +- *(1) FileScan csv [id#30839,date#30840,data#30841] Batched: false, DataFilters: [isnotnull(date#30840), (date#30840 >= 2017-10-23 00:00:00), (date#30840 <= 2019-11-18 00:00:00)], Format: CSV, Location: InMemoryFileIndex[dbfs:/mnt/partitioning/new/2017/10/1/test.csv, dbfs:/mnt/partitioning/new/2017/..., PartitionFilters: [], PushedFilters: [IsNotNull(date), GreaterThanOrEqual(date,2017-10-23 00:00:00), LessThanOrEqual(date,2019-11-18 0..., ReadSchema: struct<id:string,date:string,data:string>